## Data Loading
Data Source: https://www.kaggle.com/datasets/therohk/million-headlines

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('abcnews-date-text.csv')

data_text = data[['headline_text']]
data_text['index'] = data_text.index

documents = data_text

In [ ]:
print(len(documents))
print(documents[:5])

1244184
                                       headline_text  index
0  aba decides against community broadcasting lic...      0
1     act fire witnesses must be aware of defamation      1
2     a g calls for infrastructure protection summit      2
3           air nz staff in aust strike for pay rise      3
4      air nz strike to affect australian travellers      4


## Data Preprocessing

In [ ]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
def lemmatize_stemming(text):
    return PorterStemmer().stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [ ]:
doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['ratepayers', 'group', 'wants', 'compulsory', 'local', 'govt', 'voting']


 tokenized and lemmatized document: 
['ratepay', 'group', 'want', 'compulsori', 'local', 'govt', 'vote']


In [ ]:
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

0               [decid, commun, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

## Bag of Words on the data set

In [ ]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 commun
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [ ]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(162, 1), (240, 1), (292, 1), (589, 1), (839, 1), (3579, 1), (3580, 1)]

In [ ]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
  print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                  dictionary[bow_doc_4310[i][0]], 
                                                  bow_doc_4310[i][1]))

Word 162 ("govt") appears 1 time.
Word 240 ("group") appears 1 time.
Word 292 ("vote") appears 1 time.
Word 589 ("local") appears 1 time.
Word 839 ("want") appears 1 time.
Word 3579 ("compulsori") appears 1 time.
Word 3580 ("ratepay") appears 1 time.


## Running LDA

In [ ]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.023*"attack" + 0.023*"market" + 0.021*"adelaid" + 0.019*"death" + 0.014*"dead" + 0.013*"releas" + 0.012*"fall" + 0.012*"hotel" + 0.011*"prison" + 0.010*"kill"
Topic: 1 
Words: 0.038*"trump" + 0.024*"charg" + 0.021*"polic" + 0.021*"south" + 0.020*"news" + 0.018*"coast" + 0.017*"protest" + 0.016*"china" + 0.014*"north" + 0.013*"rise"
Topic: 2 
Words: 0.022*"world" + 0.017*"record" + 0.016*"australia" + 0.011*"melbourn" + 0.011*"street" + 0.010*"sydney" + 0.009*"talk" + 0.009*"drum" + 0.009*"beach" + 0.009*"win"
Topic: 3 
Words: 0.022*"chang" + 0.020*"govern" + 0.020*"nation" + 0.019*"elect" + 0.018*"school" + 0.018*"state" + 0.014*"plan" + 0.014*"feder" + 0.012*"announc" + 0.011*"council"
Topic: 4 
Words: 0.048*"australian" + 0.020*"open" + 0.017*"women" + 0.016*"australia" + 0.014*"leav" + 0.013*"morrison" + 0.010*"break" + 0.010*"interview" + 0.009*"game" + 0.009*"speak"
Topic: 5 
Words: 0.041*"australia" + 0.035*"case" + 0.032*"covid" + 0.025*"court" + 0.021*"live" 